In this notebook we want to show how data analysis can support an internal audit. Therefore, this notebook does not use the data as indicated in the description of the data. Furthermore, the text is in German since I want to link it to a small lecture I'll give to a German audience.

**Comments are welcome!**

## Ziel
Ziel des Notebooks ist es zu zeigen, wie Datenanalysen eine Prüfung der Revision unterstützen können. Für eine mittelgroße Prüfung (30 - 50 PT) sollte erfahrungsgemäß eine initiale Datenanalyse, wie sie im Folgenden dargestellt ist, nicht mehr als **rund 3 - 5 Tage** in Anspruch nehmen. Diese Herangehensweise ist für eine **große Menge an Fällen** anwendbar und benötigt lediglich
* einen Zugang zu mit dem Prüfungsinhalt verknüpften Daten, 
* Standardtechniken in einer für Datenanalysen geeigneten Sprache (z.B. R oder Python) und
* ein grundsätzliches Verständnis für die Daten/gesunden Menschenverstand.

Wir führen die Datenanalyse für auf Kaggle verfügbare Incident-Daten durch. Die Methoden kann man analog aber auch für Kreditdaten, Zahlungsverkehrdaten, Personaldaten und viele andere Datenarten anwenden.

In this notebook we want to show how data analysis can support an internal audit. Therefore, this notebook does not use the data as indicated in the description of the data. Furthermore, the text is in German since I want to link it to a small lecture I'll give to a German audience.

**Comments are welcome!**

In this notebook we want to show how data analysis can support an internal audit. Therefore, this notebook does not use the data as indicated in the description of the data. Furthermore, the text is in German since I want to link it to a small lecture I'll give to a German audience.

**Comments are welcome!**

In this notebook we want to show how data analysis can support an internal audit. Therefore, this notebook does not use the data as indicated in the description of the data. Furthermore, the text is in German since I want to link it to a small lecture I'll give to a German audience.

**Comments are welcome!**

In this notebook we want to show how data analysis can support an internal audit. Therefore, this notebook does not use the data as indicated in the description of the data. Furthermore, the text is in German since I want to link it to a small lecture I'll give to a German audience.

**Comments are welcome!**

## 0. Vorbemerkung
Zentral für eine gute Prüfung sind:
* ein klares Prüfungsziel, das sich aus dem Risiko des Prüffelds ableitet,
* hieraus abgeleitete Prüfungsfragen und
* ein gutes Verständnis der zugrunde liegenden Prozesse, aus denen die Daten stammen.

All dies ist für die vorliegenden, auf Kaggle öffentlich verfügbaren Daten **nicht** der Fall. Wir machen daher gewisse **Annahmen**, um dennoch eine stringente Analyse simulieren zu können.
* Ziel der Prüfung ist die Prüfung der Wirksamkeit und Ordnungsmäßigkeit des Incident-Prozesses
* Prüfungsfragen sind:
    * In wie weit ist der Prozess geeignet, Incidents in Abhängigkeit von der Kritikalität zeitgerecht zu schließen?
    * In wie weit ist die Bearbeitungsqualität zufriedenstellend?
* Bzgl. Prozess nehmen wir an, es handelt sich um einen typischen Incident-Prozess (gem. Beschreibung der Daten handelt es sich um "data gathered from the audit system of an instance of the ServiceNowTM platform used by an IT company")

Ziel der Datenanalyse ist es einerseits, übergreifende Auswertungen zu den obigen Fragen zu generieren. Typischerweise hat das Incident-Management sich hier auch selbst Ziele gesetzt, gegen die man prüfen kann - dies kann in dem Beispiel nicht betrachtet werden. Andererseits sollen Ausreißer identifiziert werden. Diese würden dann in der Praxis näher untersucht, um hier potenzielle Schwachstellen im Prozessdesign zu identifizieren. In dem hier aufgeführten Beispiel kann natürlich nur die Definition der risikoorientierten Stichprobe dargestellt werden. 

## 1. Vorbereitungen:
Laden der verwendeten Packages und Daten. Wir haben hier das Glück, einen fertigen Datensatz vorzufinden, und nicht verschiedene Datenquellen noch miteinander verbinden zu müssen. Im wirklichen Leben stecken in diesen Schritten oft nicht unerheblicher Aufwand.

In [ ]:
# Loading packages
library(tidyverse) # metapackage of all tidyverse packages
library(plotly) # Für interaktive Grafiken


In [ ]:
# Loading data
incident_event_log <- read.csv("../input/incident-response-log/incident_event_log.csv")
description <- readLines("../input/incident-response-log/Incident_response.txt")

Die Warnung können wir nach einem Blick in die Datei ignorieren. Neben den eigentlichen Daten liegt eine kurze Beschreibung vor, die wir ausgeben:

In [ ]:
print(description[-2:-1]) # lines 1 and 2 without information about variables

Als nächstes ein kurzer Blick in die Daten:

In [ ]:
head(incident_event_log)

Offenbar gibt es pro Incident mehrere Zeilen. Die jeweils finalen Einträge scheinen dann in der letzten Zeile zu stehen, wobei auch in den Zeilen davor schon manche Einträge stehen, die man hier vielleicht nicht erwartet hätte (z.B. bei Ticketeröffnung die Spalte "resolved_by"). Hier auch nochmal ein Grafik, die die Einträge je Ticketnummer zeigt:

In [ ]:
incident_event_log$number %>% table() %>% hist(main = "Anzahl Einträge je Incident")

Wir möchten uns auf die letzte Zeile je Incident konzentrieren. Dies sollte die Zeile sein, in der der Status Closed ist. Wir hätten erwartet, dass die äquivalent dazu ist, dass die Spalte "active" den Status "false" hat. Dies stimmt grundsätzlich, für eine Zeile allerdings nicht. 

In [ ]:
incident_event_log %>% filter(incident_state == "Closed" & active == "true") %>% nrow()
incident_event_log %>% filter(incident_state != "Closed" & active == "false")
incident_event_log %>% filter(number == "INC0018594")


**Beobachtung 1:** Der "INC0018594" hat (als einziger Incident) zwischendurch den Status "incidents_state" ungleich "Closed" und "active" gleich false. 

Im Folgenden reduzieren wir den Datensatz auf diejenigen Zeilen, in denen der incident_state den Status "Closed" hat. 

In [ ]:
incident_event_log_save <- incident_event_log
incident_event_log <- incident_event_log %>% filter(incident_state == "Closed")

Wir überprüfen noch, ob wir tatsächlich für jeden Incident jetzt eine Zeile haben:

In [ ]:
doppelter_name <- incident_event_log$number[duplicated(incident_event_log$number)]
print(paste("Anzahl der Incidents, die mehrfach vorkommen: ", length(doppelter_name)))
print(paste("Anzahl Incidents im Status closed gesamt: ", length(unique(incident_event_log$number))))


Angesichts der Größenverhältnisse und der zeitlichen Restriktionen widerstehen wir dem Drang, nach der Ursache der mehrfach vorkommenden Incidents zu suchen. Vielmehr merken wir sie uns für später für eine Stichprobe vor.

## 2. Univariate Ausreißeranalyse (Bearbeitungsqualität)
Für die **Bearbeitungsqualität** könnten z.B. folgende Kennzahlen Hinweise geben: 
* reassignment_count: number of times the incident has the group or the support analysts changed;
* reopen_count: number of times the incident resolution was rejected by the caller
* Bearbeitungszeit

Wir schauen uns die entsprechenden Histogramme einmal an. Zunächst für reassignment_count und reopen_count:

In [ ]:
incident_event_log %>% ggplot(aes(reassignment_count)) + geom_histogram()
incident_event_log %>% ggplot(aes(reopen_count)) + geom_histogram()

Die Bearbeitungszeit muss zunächst noch formatiert werden:

In [ ]:
incident_event_log$h <- as.POSIXct(incident_event_log$resolved_at, format="%d/%m/%Y %H:%M")
incident_event_log$h2 <- as.POSIXct(incident_event_log$opened_at, format="%d/%m/%Y %H:%M")
incident_event_log$erledigungszeit <- difftime(incident_event_log$h, incident_event_log$h2, unit = "days")
incident_event_log %>% ggplot(aes(erledigungszeit)) + geom_histogram()

Offensichtlich sind die Grafiken nicht hübsch und es kommt eine Warnung. **Meine dringende Empfehlung ist aber, nicht zeitaufwendig die Grafiken zu verschönern.** Stattdessen kann man die Ergebnisse gut ablesen: Es gibt einzelne Tickets mit hohen Assignment-Scores (> 10) und auch mit reopen_count > 2.
Wir legen eine Stichprobe an, in die wir diese Tickets speichern. Die Grenzen legen wir auf Basis der Grafiken nach Expertenschätzung fest:

In [ ]:
stichprobe <- incident_event_log %>% filter(reassignment_count > 10)
print(paste("Anzahl Zeilen: ", nrow(stichprobe)))
stichprobe$grund <- "Reassignment hoch"

In [ ]:
h <- incident_event_log %>% filter(reopen_count > 2)
print(paste("Anzahl Zeilen: ",nrow(h)))
h$grund <- "Reopen hoch"
stichprobe <- rbind(stichprobe, h)

Auch für die Bearbeitungszeit können wir die Grenzen für eine Stichprobe nach Expertenschätzung festlegen. Alternativ kann man auch die 0,1% der Fälle nehmen, die am längsten dauern:

In [ ]:
hohe_erledigungszeit <- quantile(incident_event_log$erledigungszeit, 0.999, na.rm = T)
print(paste("Hohe Erledigungszeit:", hohe_erledigungszeit))
h <- incident_event_log %>% filter(erledigungszeit > hohe_erledigungszeit)
print(paste("Anzahl Zeilen: ",nrow(h)))
h$grund <- "Erledigungszeit hoch"
stichprobe <- rbind(stichprobe, h)

Zusätzlich betrachten wir an dieser Stelle noch, ob/warum wir NA-Einträge in den Erledigungszeiten haben:

In [ ]:
print(paste("Anzahl NA-Einträge bei Erledigungszeiten:",incident_event_log %>% filter(is.na(erledigungszeit)) %>% nrow()))
print("Einträge in der Spalte resolved_at in diesem Fall:")
incident_event_log %>% filter(is.na(erledigungszeit)) %>% select(resolved_at) %>% table()
print("Einträge in der Spalte resolved_by in diesem Fall:")
incident_event_log %>% filter(is.na(erledigungszeit)) %>% select(resolved_by) %>% table()


Es ist hier also nur ein "?" in der Spalte "resolved_at" eingetragen, weshalb natürlich auch keine Erledigungszeit berechnet werden kann. In den meisten Fällen ist allerdings jemand eingetragen, der den Incident erledigt hat.

**Beobachtung 2**: Es gibt fehlende Daten beim Datum, wann ein Incident erledigt wurde.

Wir wählen eine Stichprobe, wobei wir auch Fälle betrachten, bei denen die erledigende Person unbekannt ist.

In [ ]:
h <- incident_event_log %>% filter(is.na(erledigungszeit)) %>% filter(resolved_by == "?")
h <- h[sample(1:nrow(h), 10),] # take sample of 10 elements
h$grund <- "resolved_at und resolved_by fehlend"
stichprobe <- rbind(stichprobe, h)

h <- incident_event_log %>% filter(is.na(erledigungszeit)) %>% filter(resolved_by != "?")
h <- h[sample(1:nrow(h), 10),] # take sample of 10 elements
h$grund <- "resolved_at fehlend"
stichprobe <- rbind(stichprobe, h)

Zusätzlich wollten wir ja, wie in Kapitel 1 angemerkt, mehrfach vorkommende Incidents in die Stichprobe nehmen: 

In [ ]:
h <- incident_event_log %>% filter(number %in% doppelter_name)
h <- h[sample(1:nrow(h), 10),] # take sample of 10 elements
h$grund <- "Incident mehrfach"
stichprobe <- rbind(stichprobe, h)

## 3. Kritikalität
Ein Prüfungsfrage lautete gem. Abschnitt 0: *In wie weit ist der Prozess geeignet, Incidents in Abhängigkeit von der Kritikalität zeitgerecht zu schließen?*

Entsprechend ist zunächst auszuwerten, welche Incidents als kritisch angesehen werden können. 

Auf eine hohe Kritikalität der Incidents könnten z.B. folgende Variablen hinweisen:
* impact: description of the impact caused by the incident (values: 1â€“High; 2â€“Medium; 3â€“Low);
* urgency: description of the urgency informed by the user for the incident resolution (values: 1â€“High; 2â€“Medium; 3â€“Low);
* priority: calculated by the system based on 'impact' and 'urgency';
* made_sla: boolean attribute that shows whether the incident exceeded the target SLA;

Wobei die letzte Variable ohne weiteres Wissen schwer zu beurteilen ist, ggf. kann hier auch die Erledigungszeit eine Rolle spielen.

Interessant ist auch, für welche Incidents eine Bestätigung der Priorität vorgenommen wurde. 

Im Folgenden zeigen wir die Verteilung der Incidents auf diese Variablen durch Histogramme. 


In [ ]:
incident_event_log %>% ggplot(aes(impact, fill = u_priority_confirmation)) + geom_histogram(stat="count")
incident_event_log %>% ggplot(aes(urgency, fill = u_priority_confirmation)) + geom_histogram(stat="count")
incident_event_log %>% ggplot(aes(priority, fill = u_priority_confirmation))+ geom_histogram(stat="count")
incident_event_log %>% ggplot(aes(made_sla, fill=priority)) + geom_histogram(stat="count")

Auch hier widerstehen wir der Versuchung, die Grafen z.B. durch schönere Anordnung (grid.arrange, plot_grid) hübscher zu machen. Stattdessen schauen wir uns die Grafen genauer an:

**Beobachtung 3:** Die große Mehrheit wird Medium/Moderate geratet. Es sollte überprüft werden, ob die entsprechenden Einstufungsprozesse angemessen sind.

Die Bestätigung der Priorität findet wie erwartet insb. bei hoch priorisierten Incidents statt. In die Stichprobe nehmen wir ein paar Fälle auf, die hoch priorisiert sind, bei denen es aber dennoch keine Überprüfung gab. 

In [ ]:
h <- incident_event_log %>% filter(priority == "1 - Critical") %>% filter(u_priority_confirmation == "false")
h
h$grund <- "Kritische Priorität nicht überprüft"
stichprobe <- rbind(stichprobe, h)

Wie erwartet ist made_sla eher bei kritischen Incidents "false". 

**Beobachtung 4:** Es gibt niedrig priorisierte Incidents, die dennoch zu einer Nichterfüllung des SLA führen.

Wir nehmen hiervon gleich eine Stichprobe:

In [ ]:
h <- incident_event_log %>% filter(priority == "4 - Low") %>% filter(made_sla == "false")
nrow(h)

121 Fälle sind uns zu viel für die Stichprobe, daher ziehen wir hier noch eine Zufallsauswahl.

In [ ]:
h <- h[sample(1:nrow(h), 20),] # take sample of 20 elements
h$grund <- "Niedrig priorisierter Incident bei Nichterfüllung SLA"
stichprobe <- rbind(stichprobe, h)

## 4. Kritikalität und Bearbeitungszeit

Die erste Prüfungsfrage lautete gem. Abschnitt 0: In wie weit ist der Prozess geeignet, Incidents in Abhängigkeit von der Kritikalität zeitgerecht zu schließen?

Wenn wir annehmen, dass die Priorität die Kritikalität am besten abbildet, liefert folgender Boxplot eine gute Annäherung an die Frage:

In [ ]:
incident_event_log %>% ggplot(aes(priority, erledigungszeit)) + geom_boxplot()


**Beobachtung 5:** Kritischere Incidents werden im Schnitt nicht früher geschlossen, als weniger kritische. Der Prozess ist hier auf die Steuerungswirkung der Priorität zu überprüfen.

Wir betrachten das nochmal in Zahlen:

In [ ]:
incident_event_log %>% group_by(priority) %>% summarise(Durchschnitt = mean(erledigungszeit, na.rm=T), Median = median(erledigungszeit, na.rm=T))

Auch hier nehmen wir die kritischen mit der höchsten Bearbeitungszeit in die Stichprobe.

In [ ]:
h <- incident_event_log %>% filter(priority == "1 - Critical") %>% filter(erledigungszeit > 100)
h$grund <- "Kritischer Incident mit hoher Erledigungszeit"
stichprobe <- rbind(stichprobe, h)

## 5. Wechsel der Priorität im Verlauf
In den vorhergehenden Auswertungen haben wir implizit angenommen, dass die Priorität über den Incident unverändert war. Es ist aber durchaus möglich, dass z.B. aufgrund von langen Bearbeitungszeiten ein Incident seine Kritikalität ändert. Wir untersuchen das im Folgenden für kritische Incidents.

In [ ]:
# Alle kritischen Incidents:
kritische_incidents <- incident_event_log %>% filter(priority == "1 - Critical")

# Alle am Ende kritischen Incidents mit allen Zeilen davor
kritische_incidents_alle_eintraege <- incident_event_log_save %>% filter(number %in% kritische_incidents$number)

# Incidents, die am Ende kritisch waren, zwischendurch aber nicht
kritische_incidents_zwischendurch_andere_priority <- kritische_incidents_alle_eintraege %>% filter(priority != "1 - Critical")
print(paste("Anzahl Incidents, die am Ende kritisch waren, zwischendurch aber nicht: ", length(unique(kritische_incidents_zwischendurch_andere_priority$number))))

# Incident numbers, die immer kritisch waren
immer_kritisch <- setdiff(kritische_incidents$number, kritische_incidents_zwischendurch_andere_priority$number)

# Incidents (alle Spalten), die immer kritisch waren
immer_kritische_incidents <- incident_event_log %>% filter(number %in% immer_kritisch)

# Mean und Median dieser Incidents
print(paste("Durchschnittliche Erledigungszeit der Incidents, die immer kritisch waren: ", mean(immer_kritische_incidents$erledigungszeit)))
print(paste("Median der Erledigungszeit der Incidents, die immer kritisch waren: ", median(immer_kritische_incidents$erledigungszeit)))

Man sieht, dass die Ergebnisse besser sind, wenn die Incidents von Anfang an kritisch waren. Dennoch ist die Durchschnittszeit noch höher als bei High und nicht niedriger als bei Moderate und der Median höher als bei Moderate und Low.

## 6. Multivariate Analyse
Wir haben oben bereits einzelne Analysen gemacht, die mehrere Variablen beinhalteten (z.B. Erledigungszeit in Abhängigkeit von der Kritikalität). In diesem Kapitel möchten wir aber zeigen, wie man mit einer relativ großen Menge an Variablen umgehen kann. 

Zunächst einmal müssen wir alle relevanten Variablen in numerische Variablen umwandeln. Der Einfachheit halber wandeln wir dabei auch ordinale Variablen, als seien sie metrisch (d.h. z.B. der Abstand zwischen "Critical" und "High" ist der gleiche wie zwischen "High" und "Moderate").

Zusätzlich entfernen wir die Incidents, deren Erledigungszeit NA ist.

Output ist eine Zusammenfassung der verschiedenen Variablen.

In [ ]:
df <- incident_event_log
df <- df %>% select(priority, reassignment_count, reopen_count, erledigungszeit, made_sla, u_priority_confirmation)
df$priority[df$priority == "1 - Critical"] <- 3
df$priority[df$priority == "2 - High"] <- 2
df$priority[df$priority == "3 - Moderate"] <- 1
df$priority[df$priority == "4 - Low"] <- 0
df$priority <- as.numeric(df$priority)

df$made_sla[df$made_sla == "false"] <- 0
df$made_sla[df$made_sla == "true"] <- 1
df$made_sla <- as.numeric(df$made_sla)

df$u_priority_confirmation[df$u_priority_confirmation == "false"] <- 0
df$u_priority_confirmation[df$u_priority_confirmation == "true"] <- 1
df$u_priority_confirmation <- as.numeric(df$u_priority_confirmation)

df$erledigungszeit <- as.numeric(df$erledigungszeit)

df <- df %>% filter(!is.na(erledigungszeit))

summary(df)

### 6.1 Principal Component-Analyse
Wir führen nun eine Principal Component-Analyse durch. Dies ist eine der gängigsten Methoden, um hochdimensionale Daten mit möglichst wenig Informationsverlust zweidimensional anzuzeigen. Die Methode hat natürlich auch Nachteile/Einschränkungen (insb., dass lediglich lineare Transformationen möglich sind und die optimale Wahl der Achsen bzw. der Informationsverlust nur unter strikten Bedingungen bewiesen werden kann), worauf wir hier aber nicht eingehen. Allerdings macht es durchaus Sinn zu prüfen, wie viel "Prozent der Information" man mit zwei Achsen sehen kann.

In [ ]:
df.pca <- prcomp(df, center = TRUE, scale. = TRUE)
summary(df.pca)

Mit zwei Achsen kann man also rund 47% der Informationen sehen, was bei 6 Dimensionen kein besonders hoher Wert ist (bei zufälligen Achsen wären es im Schnitt 33%).

Wir plotten nun die beiden ersten Achsen.

In [ ]:
# Wandle df.pca in data.frame um
df.pca_data_frame <- df.pca[["x"]] %>% data.frame()

# Speichere incident-Namen
namen <- incident_event_log %>% filter(!is.na(erledigungszeit))
namen <- namen$number

#plot
plot_ly(df.pca_data_frame, x=~PC1, y=~PC2, text=namen, type="scatter")

Die einzelnen Punkte am Rand betrachten wir als Ausreißer. Als pragmatischen Ansatz fahren wir mit der Maus über diese Punkte und schreiben sie in einen Vektor. Wir zeichnen die Grafik zur Kontrolle bzw. zur besseren Nachvollziehbarkeit nochmal (diesmal ohne interaktives Element) und markieren dabei die Elemente des Vektors farblich.

In [ ]:
ausreisser <- c("INC0012499", "INC0007521", "INC0005927", "INC0015902", "INC0002129","INC0007521", "INC0002780" , "INC0011665", "INC0001929", "INC0007593","INC0002483","INC0003419", "INC0007229", "INC0019396", "INC0011206", "INC0019595", "INC0003982", "INC0004210", "INC0020718", "INC0007349", "INC0012815", "INC0019131")
df.pca_data_frame$namen <- namen
df.pca_data_frame$ausreisser <- ifelse(namen %in% ausreisser, 1, 0)
df.pca_data_frame %>% ggplot(aes(PC1, PC2, colour=as.factor(ausreisser))) + geom_point()

Die wesentliche Frage ist: Haben wir durch diese Analyse neue Ausreißer entdeckt, die wir in der univariaten Analse noch nicht entdeckt hatten. 

In [ ]:
print(paste("Anzahl multivariater Ausreißer: ", length(unique(ausreisser))))
h <- setdiff(ausreisser, stichprobe$number)
print(paste("Anzahl multivariater Ausreißer, die nicht in der univariaten Ausreißeranalyse identifiziert wurden: ", length(h)))



4 von 21 Ausreißer sind also "neu". Die multivariate Analyse liefert also neue Informationen, allerdings nicht besonders viele.

In [ ]:
h <- incident_event_log %>% filter(number %in% ausreisser)
h$grund <- "Ausreißer gem. PCA"
stichprobe <- rbind(stichprobe, h)

Natürlich ist eine solche, rein graphische Analyse angreifbar (wobei es nur um eine Stichprobenermittlung geht). Es gibt natürlich auch Algorithmen, die ähnliche Analysen vornehmen, zum Beispiel auf Basis von k-nearest neighbourhoods.

### 6.2 Autoencoder
Wir haben oben ein paar Schwächen der Principal Component Analyse angesprochen. Deshalb verwenden wir hier noch ein zweites, nichtlineares Verfahren zur Ausreißeranalyse. Wir trainieren ein kleines neuronales Netz, das die Daten möglichst gut kompimieren soll. Diejenigen Datensätze, bei denen die Komprimierung weit entfernt vom Original ist, sind Ausreißer.

Zunächst bereiten wir die Daten und Pakete vor. 

In [ ]:
# Bibliothek laden
library(autoencoder)
library(scales)

# Variablen skalieren
df_scaled <- sapply(df, rescale)
summary(df_scaled)

Jetzt werfen wir das Modell an. Gemäß Package ist das Modell insbesondere für "sparse Autoencoders" (um bei vielen Variablen Overfitting zu vermeiden). Das ist hier nicht notwendig, weshalb wir den Parameter beta (weight of sparsity penalty term) auf 0 setzen. Selbstverständlich kann man hier auch andere Packages verwenden. 

In [ ]:
# Berechne Modell
set.seed(123)
model <- autoencode(df_scaled, beta = 0, rho = 0.01, epsilon = 0.001, N.hidden = 3, lambda = 0.0002, rescale.flag = FALSE)
# Berechne Vorhersage des Modells
ergebnis <- predict(model, df_scaled, hidden.output = FALSE)

Wir berechnen nun die Abweichungen zwischen Modell-Vorhersage und Ist-Werten und zeichnen diese in ein Histogramm. Dabei markieren wir diejenigen Incidents, die bereits in der Stichprobe sind:

In [ ]:
# Berechne die Abweichung zwischen der Vorhersage und den Ist-Werten (je Variable und je Incident)
abweichung <- as.matrix(df_scaled) - ergebnis$X.output
# Berechne das Quadrat hiervon
abweichung_2 <- abweichung^2
# Berechne die Summe über alle quadratischen Abweichungen je Incident
abweichung_gesamt <- rowSums(abweichung_2)
# Nehme diese mit den Namen in ein Dataframe
abweichung_df <- data.frame(abweichung_gesamt, namen)
colnames(abweichung_df) <- c("Abweichung", "IncidentName")
abweichung_df$Ist_in_Stichprobe <- ifelse(abweichung_df$IncidentName %in% stichprobe$number, "ja", "nein")

# Zeichne Histogramm
g <- abweichung_df %>% ggplot(aes(Abweichung, fill = Ist_in_Stichprobe)) + geom_histogram()
ggplotly(g)


Es sind im Vergleich zu allen Incidents nur sehr wenige in der Stichprobe, weshalb man sie im Diagramm nicht erkennt. Wählt man aber einen Bereich von ca. Count 0 - 50, Abweichung 0,5 - 1,1 aus, erkennt man gut, dass Incidents mit der höchsten Abweichung schon in der Stichprobe enthalten sind. Ab ca. einer Abweichung von 0,5 sind aber auch Incidents vorhanden, die noch nicht in der Stichprobe sind. Wir setzen die Grenze zur Einbindung daher auf 0,4.

In [ ]:
hh <- abweichung_df %>% filter(Abweichung > 0.4)
print(paste("Anzahl Incidents mit Abweichung > 0,4: ",length(unique(hh$IncidentName))))
stichprobe_in_abweichung <- stichprobe %>% filter(number %in% hh$IncidentName)
print(paste("Davon bereits in der Stichprobe enthalten: ", length(unique(stichprobe_in_abweichung$number))))

Wir fügen die Punkte der Stichprobe hinzu:

In [ ]:
h <- incident_event_log %>% filter(number %in% hh$IncidentName)
h$grund <- "Ausreißer gem. Autoencoder"
stichprobe <- rbind(stichprobe, h)

## Abschluss
Wir haben in der kurzen Analyse wichtige Informationen für unsere Prüfung erhalten:
* Ein grundsätzliches Verständnis für das Mengengerüst des Prozesses
* Mehrere **Beobachtungen**, die Hinweise auf Prozessschwächen sein können
* Eine **risikoorientierte Stichrpobe**, die Ausgangspunkt für weitere Untersuchungen sein kann

Da für die Analyse nur ungefähr 10% der veranschlagten Zeit verwendet wurden, ist auch noch genug Zeit, diesen Punkten nachzugehen. Selbstverständlich könnte man noch viel mehr Analysen auf diesen Daten machen, aber wird es erfahrungsgemäß schwer, allen Auffälligkeiten tatsächlich in der gebotenen Genauigkeit nachzugehen.

Wir schließen mit einem kurzen Blick auf die Verteilung der Stichprobe nach verschiedenen Gründen. Es macht Sinn, die Tickets zuerst zu betrachten, die aus mehrfachen Grund in der Stichprobe sind:

In [ ]:
stichprobe$grund %>% table()
Haeufigkeit_Incident_in_Stichprobe <- stichprobe %>% group_by(number) %>% summarise(count = n())
hist(Haeufigkeit_Incident_in_Stichprobe$count)

Und jetzt geht die Detailarbeit los...